#OpenCV2 HaarCascade

In [1]:
import cv2
import time
import psutil
import GPUtil

# Muat Haar Cascade untuk deteksi wajah
face_cascade = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')

# Periksa apakah file Haar Cascade berhasil dimuat
if face_cascade.empty():
    print("Error: Tidak dapat memuat file Haar Cascade.")
    exit()

# Inisialisasi kamera (biasanya 0 adalah kamera default, Anda mungkin perlu mengubahnya jika ada beberapa kamera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Tidak dapat membuka kamera.")
    exit()

# Set resolusi kamera ke 1280x720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Variabel untuk menghitung FPS
prev_time = 0

# Fungsi untuk mendapatkan penggunaan GPU
def get_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        return gpu.load * 100  # Mengembalikan penggunaan GPU dalam persentase
    return 0

while True:
    # Baca frame dari kamera
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Tidak dapat membaca frame dari kamera.")
        break
    
    # Konversi frame ke skala abu-abu
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Deteksi wajah
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Gambar persegi panjang di sekitar wajah yang terdeteksi
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2)
        
        # Hitung titik tengah bounding box
        center_x = x + w // 2
        center_y = y + h // 2
        
        # Tampilkan informasi bounding box dan titik tengah
        debug_text = f"Box: ({x}, {y}), W: {w}, H: {h}, Center: ({center_x}, {center_y})"
        #print(debug_text)  # Tampilkan di console/serial monitor
        
        # Tampilkan informasi pada frame
        cv2.putText(frame, debug_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 1, cv2.LINE_AA)
        cv2.circle(frame, (center_x, center_y), 5, (0, 165, 255), -1)
    
    # Dapatkan waktu sekarang
    curr_time = time.time()
    exec_time = curr_time - prev_time
    prev_time = curr_time
    fps = 1 / exec_time
    
    # Dapatkan penggunaan CPU dan GPU
    cpu_usage = psutil.cpu_percent()
    gpu_usage = get_gpu_usage()
    
    # Tambahkan teks ke frame
    cv2.putText(frame, 'Face Detection OpenCV2', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'CPU Usage: {cpu_usage}%', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'GPU Usage: {gpu_usage:.1f}%', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    
    # Tampilkan frame dengan wajah yang terdeteksi dan data debug
    cv2.imshow('Face Detection OpenCV2', frame)
    
    # Tekan 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan setelah selesai
cap.release()
cv2.destroyAllWindows()

#Yolov8

In [2]:
import cv2
import time
import psutil
import GPUtil
from ultralytics import YOLO

# Muat model YOLOv8
model = YOLO('../yolov8n-face.pt')  # model YOLOv8 yang dilatih untuk deteksi wajah

# Inisialisasi kamera (biasanya 0 adalah kamera default, Anda mungkin perlu mengubahnya jika ada beberapa kamera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Tidak dapat membuka kamera.")
    exit()

# Set resolusi kamera ke 1280x720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Variabel untuk menghitung FPS
prev_time = 0

# Fungsi untuk mendapatkan penggunaan GPU
def get_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        return gpu.load * 100  # Mengembalikan penggunaan GPU dalam persentase
    return 0
    
while True:
    # Baca frame dari kamera
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Tidak dapat membaca frame dari kamera.")
        break
    
    # Deteksi wajah menggunakan YOLOv8
    results = model(frame)

    # Proses hasil deteksi
    for result in results:
        boxes = result.boxes  # Akses bounding box dari hasil deteksi
        for box in boxes:
            # Dapatkan koordinat bounding box dan konversi ke integer
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
            conf = box.conf.item()  # Dapatkan confidence score

            # Gambar persegi panjang di sekitar wajah yang terdeteksi
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 165, 255), 2)
            
            # Hitung titik tengah bounding box
            center_x = (x_min + x_max) // 2
            center_y = (y_min + y_max) // 2
            
            # Tampilkan informasi bounding box dan titik tengah
            debug_text = f"Box: ({x_min}, {y_min}), W: {x_max-x_min}, H: {y_max-y_min}, Center: ({center_x}, {center_y})"
            #print(debug_text)  # Tampilkan di console/serial monitor
            
            # Tampilkan informasi pada frame
            cv2.putText(frame, debug_text, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 165, 255), 1, cv2.LINE_AA)
            cv2.circle(frame, (center_x, center_y), 5,(0, 165, 255), -1)
    
    # Dapatkan waktu sekarang
    curr_time = time.time()
    exec_time = curr_time - prev_time
    prev_time = curr_time
    fps = 1 / exec_time
    
    # Dapatkan penggunaan CPU dan GPU
    cpu_usage = psutil.cpu_percent()
    gpu_usage = get_gpu_usage()
    
    # Tambahkan teks ke frame
    cv2.putText(frame, 'Face Detection Yolov8', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'CPU Usage: {cpu_usage}%', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'GPU Usage: {gpu_usage:.1f}%', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    
    # Tampilkan frame dengan wajah yang terdeteksi dan data debug
    cv2.imshow('Face Detection Yolov8', frame)
    
    # Tekan 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan setelah selesai
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 face, 97.5ms
Speed: 3.0ms preprocess, 97.5ms inference, 91.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 3.9ms
Speed: 2.0ms preprocess, 3.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 4

In [ ]:
#YOLOv10

In [18]:
import cv2
import time
import psutil
import GPUtil
from ultralytics import YOLOv10

# Muat model YOLOv8
model = YOLOv10('../yolov10s-face.pt')  # model YOLOv10 yang dilatih untuk deteksi wajah

# Inisialisasi kamera (biasanya 0 adalah kamera default, Anda mungkin perlu mengubahnya jika ada beberapa kamera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Tidak dapat membuka kamera.")
    exit()

# Set resolusi kamera ke 1280x720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Variabel untuk menghitung FPS
prev_time = 0

# Fungsi untuk mendapatkan penggunaan GPU
def get_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        return gpu.load * 100  # Mengembalikan penggunaan GPU dalam persentase
    return 0
    
while True:
    # Baca frame dari kamera
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Tidak dapat membaca frame dari kamera.")
        break
    
    # Deteksi wajah menggunakan YOLOv10
    results = model(frame)[0]

    # Proses hasil deteksi
    for result in results:
        boxes = result.boxes  # Akses bounding box dari hasil deteksi
        for box in boxes:
            # Dapatkan koordinat bounding box dan konversi ke integer
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
            conf = box.conf.item()  # Dapatkan confidence score

            # Gambar persegi panjang di sekitar wajah yang terdeteksi
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 165, 255), 2)
            
            # Hitung titik tengah bounding box
            center_x = (x_min + x_max) // 2
            center_y = (y_min + y_max) // 2
            
            # Tampilkan informasi bounding box dan titik tengah
            debug_text = f"Box: ({x_min}, {y_min}), W: {x_max-x_min}, H: {y_max-y_min}, Center: ({center_x}, {center_y})"
            #print(debug_text)  # Tampilkan di console/serial monitor
            
            # Tampilkan informasi pada frame
            cv2.putText(frame, debug_text, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 1, cv2.LINE_AA)
            cv2.circle(frame, (center_x, center_y), 5, (0, 165, 255), -1)
    
    # Dapatkan waktu sekarang
    curr_time = time.time()
    exec_time = curr_time - prev_time
    prev_time = curr_time
    fps = 1 / exec_time
    
    # Dapatkan penggunaan CPU dan GPU
    cpu_usage = psutil.cpu_percent()
    gpu_usage = get_gpu_usage()
    
    # Tambahkan teks ke frame
    cv2.putText(frame, 'Face Detection Yolov10', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'CPU Usage: {cpu_usage}%', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'GPU Usage: {gpu_usage:.1f}%', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
    
    # Tampilkan frame dengan wajah yang terdeteksi dan data debug
    cv2.imshow('Face Detection Yolov10', frame)
    
    # Tekan 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan setelah selesai
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 face, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 8.0m